In [1]:
import clingo
import numpy as np
import torch
import matplotlib.pyplot as plt
import pprint
pp = pprint.PrettyPrinter(indent=4)

# ASP Playground
This notebook showcases stable model computation for different logic programs used in this repo.

In [2]:
class Context:
    def id(self, x):
        return x
    def seq(self, x, y):
        return [x, y]
    
    def on_model(m):
        print(m)

# MNIST Addition Task

In [3]:
program = """
1{digit(0,i1,0); digit(0,i1,1); digit(0,i1,2); digit(0,i1,3); digit(0,i1,4); digit(0,i1,5); digit(0,i1,6); digit(0,i1,7); digit(0,i1,8); digit(0,i1,9)}1.
1{digit(0,i2,0); digit(0,i2,1); digit(0,i2,2); digit(0,i2,3); digit(0,i2,4); digit(0,i2,5); digit(0,i2,6); digit(0,i2,7); digit(0,i2,8); digit(0,i2,9)}1.
img(i1). img(i2).
addition(A,B,N) :- digit(0,A,N1), digit(0,B,N2), N=N1+N2.
:- not addition(i1, i2, 3).
"""
ctl = clingo.Control(["0", "--warn=none"])
ctl.add("base", [], program)
ctl.ground([("base", [])], context=Context())
out = ctl.solve(on_model=Context.on_model)

pp.pprint(ctl.statistics['summary'])

digit(0,i1,0) digit(0,i2,3) addition(i1,i1,0) addition(i2,i1,3) addition(i1,i2,3) addition(i2,i2,6) img(i1) img(i2)
digit(0,i1,2) digit(0,i2,1) addition(i1,i1,4) addition(i2,i1,3) addition(i1,i2,3) addition(i2,i2,2) img(i1) img(i2)
digit(0,i1,3) digit(0,i2,0) addition(i1,i1,6) addition(i2,i1,3) addition(i1,i2,3) addition(i2,i2,0) img(i1) img(i2)
digit(0,i1,1) digit(0,i2,2) addition(i1,i1,2) addition(i2,i1,3) addition(i1,i2,3) addition(i2,i2,4) img(i1) img(i2)
{   'call': 0.0,
    'concurrency': 1.0,
    'costs': [],
    'exhausted': 1.0,
    'lower': [],
    'models': {'enumerated': 4.0, 'optimal': 0.0},
    'result': 1.0,
    'signal': 0.0,
    'times': {   'cpu': 0.002699000000000007,
                 'sat': 1.71661376953125e-05,
                 'solve': 0.0007531642913818359,
                 'total': 0.0023660659790039062,
                 'unsat': 0.00015401840209960938},
    'winner': 0.0}


# Social Study Task with TabNet

In [4]:
program = """
1{wealth(p1,0); wealth(p1,1)}1.
1{wealth(p2,0); wealth(p2,1)}1.

person(p1). person(p2).

1{sex(p1,0); sex(p1,1)}1.
1{sex(p2,0); sex(p2,1)}1.

different_wealth(P1,W1,P2,W2) :- wealth(P1,W1), wealth(P2,W2), W1!=W2.
different_sex(P1,S1,P2,S2) :- sex(P1,S1), sex(P2,S2), S1!=S2.

wealth_gap(P1,S1,P2,S2) :- different_sex(P1,S1,P2,S2), different_wealth(P1,W1,P2,W2).

%:- not wealth_gap(p1,1,p2,0).
:- not wealth_gap(p1,0,p2,1).
"""

ctl = clingo.Control(["0", "--warn=none"])
ctl.add("base", [], program)
ctl.ground([("base", [])], context=Context())
out = ctl.solve(on_model=Context.on_model)
pp.pprint(ctl.statistics['summary'])

wealth(p1,0) wealth(p2,1) sex(p1,0) sex(p2,1) different_wealth(p2,1,p1,0) different_wealth(p1,0,p2,1) different_sex(p2,1,p1,0) different_sex(p1,0,p2,1) wealth_gap(p2,1,p1,0) wealth_gap(p1,0,p2,1) person(p1) person(p2)
wealth(p1,1) wealth(p2,0) sex(p1,0) sex(p2,1) different_wealth(p2,0,p1,1) different_wealth(p1,1,p2,0) different_sex(p2,1,p1,0) different_sex(p1,0,p2,1) wealth_gap(p2,1,p1,0) wealth_gap(p1,0,p2,1) person(p1) person(p2)
{   'call': 0.0,
    'concurrency': 1.0,
    'costs': [],
    'exhausted': 1.0,
    'lower': [],
    'models': {'enumerated': 2.0, 'optimal': 0.0},
    'result': 1.0,
    'signal': 0.0,
    'times': {   'cpu': 0.0023040000000000838,
                 'sat': 5.0067901611328125e-06,
                 'solve': 0.0005331039428710938,
                 'total': 0.0018379688262939453,
                 'unsat': 0.00035309791564941406},
    'winner': 0.0}


In [35]:
program = """
% tabnet(wealth(1,X), [0,1]) :- person(X).
% {"p is wealthy" OR "p is not wealthy"}
1{wealth(p,0); wealth(p,1)}1.

person(p).

wealthy(P,W) :- wealth(P,W).

%:- not wealthy(p,1).
:- not wealthy(p,0).

"""

"""
1{digit(0,i1,0); digit(0,i1,1); digit(0,i1,2); digit(0,i1,3); digit(0,i1,4); digit(0,i1,5); digit(0,i1,6); digit(0,i1,7); digit(0,i1,8); digit(0,i1,9)}1.
1{digit(0,i2,0); digit(0,i2,1); digit(0,i2,2); digit(0,i2,3); digit(0,i2,4); digit(0,i2,5); digit(0,i2,6); digit(0,i2,7); digit(0,i2,8); digit(0,i2,9)}1.
img(i1). img(i2).
addition(A,B,N) :- digit(0,A,N1), digit(0,B,N2), N=N1+N2.
:- not addition(i1, i2, 3).
"""

ctl = clingo.Control(["0", "--warn=none"])
ctl.add("base", [], program)
ctl.ground([("base", [])], context=Context())
out = ctl.solve(on_model=Context.on_model)
pp.pprint(ctl.statistics['summary'])

wealth(p,0) wealthy(p,0) person(p)
{   'call': 0.0,
    'concurrency': 1.0,
    'costs': [],
    'exhausted': 1.0,
    'lower': [],
    'models': {'enumerated': 1.0, 'optimal': 0.0},
    'result': 1.0,
    'signal': 0.0,
    'times': {   'cpu': 0.0018439999999770862,
                 'sat': 5.9604644775390625e-06,
                 'solve': 0.00012993812561035156,
                 'total': 0.0009908676147460938,
                 'unsat': 0.0001239776611328125},
    'winner': 0.0}


# SlotAttention Experiment

### 1. Assign each slot to one object name such that each object is asigned once

this yields 3!=6 results for assigning 3 objects into slots

In [34]:
ctl = clingo.Control(["0", "--warn=none"])
ctl.add("base", [], 
    """
    
    %data
    slot(1..3).
    name(1..3).
    %slot(s1).
    %slot(s2).
    %slot(s3).
    %slot(s4).
    %name(n1).
    %name(n2).
    %name(n3).
    %name(n4).
    
    % | n\s | 1     | 2     | 3     |
    % | 1   | (1,1) | (1,2) | (1,3) |
    % | 2   | (2,1) | (2,2) | (2,3) |
    % | 3   | (3,1) | (3,2) | (3,3) |
    
    % slot_name_comb(N,X) IF name(N) AND slot(X).
    % {slot_name_comb(n1,s1) XOR slot_name_comb(n1,s2) XOR slot_name_comb(n1,s3)} 
    % {slot_name_comb(N,X): name(N)}=1 IF slot(X).
    % Example for slot(s1):
    % {slot_name_comb(s1,n1); slot_name_comb(s1,n2); slot_name_comb(s1,n3)} = 1
    % {slot_name_comb(s1,n1) OR slot_name_comb(s1,n2) OR slot_name_comb(s1,n3)} pick axactly one
    % assign each slot a name of an object
    {assign_slot_name(S,N): name(N)}=1 :- slot(S).  % Generator: problem we have dublicated slots
    % {assign_slot_name(S,N): slot(S), name(N), S <= N}=1.
    % {assign_slot_name(S,N): slot(S), name(N)} = 3.
    
    % Constraint: if there are two identical slots, then they can not have different names.
    % :- assign_slot_name(N1,S1), assign_slot_name(N2,S2), S1==S2, N1!=N2.  % Tester: sort out dublicted slots
    % :- assign_slot_name(S,N), S==N.
    % :- {assign_slot_name(S,N)} != 1.
    % :- assign_slot_name(S,N), assign_slot_name(S',N), S == S'.
    % :- assign_slot_name(S,N), assign_slot_name(S,N'), N == N'.    
    % :- assign_slot_name(S,N), assign_slot_name(S',N'), (S,N) != (S',N'). 
    % :- assign_slot_name(S,N), S < N.
    """)
ctl.ground([("base", [])], context=Context())
out = ctl.solve(on_model=Context.on_model)
# print(type(out))
print(out)


pp.pprint(ctl.statistics['summary'])

slot(1) slot(2) slot(3) name(1) name(2) name(3) assign_slot_name(2,2) assign_slot_name(3,2) assign_slot_name(1,3)
slot(1) slot(2) slot(3) name(1) name(2) name(3) assign_slot_name(3,2) assign_slot_name(1,3) assign_slot_name(2,3)
slot(1) slot(2) slot(3) name(1) name(2) name(3) assign_slot_name(3,1) assign_slot_name(2,2) assign_slot_name(1,3)
slot(1) slot(2) slot(3) name(1) name(2) name(3) assign_slot_name(3,1) assign_slot_name(1,3) assign_slot_name(2,3)
slot(1) slot(2) slot(3) name(1) name(2) name(3) assign_slot_name(2,1) assign_slot_name(3,2) assign_slot_name(1,3)
slot(1) slot(2) slot(3) name(1) name(2) name(3) assign_slot_name(2,1) assign_slot_name(3,1) assign_slot_name(1,3)
slot(1) slot(2) slot(3) name(1) name(2) name(3) assign_slot_name(1,3) assign_slot_name(2,3) assign_slot_name(3,3)
slot(1) slot(2) slot(3) name(1) name(2) name(3) assign_slot_name(2,2) assign_slot_name(1,3) assign_slot_name(3,3)
slot(1) slot(2) slot(3) name(1) name(2) name(3) assign_slot_name(2,1) assign_slot_name(1

### 2. Now we assign each object name to an object 

In this example we want to assign two blue circles to the three slots. On Slot is then "left open" which means that ASP generates stable models with all possible combinations.  
Objects can have shape triangle or circle and color red and blue. Therefore we can have objects with 4 different property combinations of color and shape.

This yields 3! Slot combinations times 4 for one object that can be choosen. -> 6*4 = 24 Stable Models

In [5]:
'''
slot_name_comb(o2,s1),slot_name_comb(o1,s2) , slot_name_comb(o1,s1), slot_name_comb(o2,s2)  
we have both but want a unique assignment->
slot_name_comb(o2,s1),slot_name_comb(o1,s2) 
slot_name_comb(o1,s1), slot_name_comb(o2,s2)
'''


ctl = clingo.Control(["0", "--warn=none"])
ctl.add("base", [], 
    """
    shape(0, s1, circle); shape(0, s1, triangle).
    shape(0, s2, circle); shape(0, s2, triangle).
    shape(0, s3, circle); shape(0, s3, triangle).

    color(0, s1, red); color(0, s1, blue).
    color(0, s2, red); color(0, s2, blue). 
    color(0, s3, red); color(0, s3, blue). 

    slot(s1).
    slot(s2).
    slot(s3).
    
    name(o1).
    name(o2).
    name(o3).
    
    {slot_name_comb(N,X): slot(X) }=1 :- name(N). %problem we have dublicated slots
    :-  slot_name_comb(N1,X1), slot_name_comb(N2,X2), X1 == X2, N1 != N2. 
   
    obj(N,C,S) :- color(0, X, C), shape(0, X, S), slot(X), name(N), slot_name_comb(N,X).

    
    :- not obj(o1, blue,circle).
    :- not obj(o2, blue,circle).
    
    
    """)
ctl.ground([("base", [])], context=Context())
out =ctl.solve(on_model=Context.on_model)

pp.pprint(ctl.statistics['summary']) 

shape(0,s1,circle) shape(0,s2,triangle) shape(0,s3,circle) color(0,s1,blue) color(0,s2,red) color(0,s3,blue) slot(s1) slot(s2) slot(s3) slot_name_comb(o2,s1) slot_name_comb(o3,s2) slot_name_comb(o1,s3) name(o1) name(o2) name(o3) obj(o1,blue,circle) obj(o2,blue,circle) obj(o3,red,triangle)
shape(0,s1,circle) shape(0,s2,triangle) shape(0,s3,circle) color(0,s1,blue) color(0,s2,blue) color(0,s3,blue) slot(s1) slot(s2) slot(s3) slot_name_comb(o2,s1) slot_name_comb(o3,s2) slot_name_comb(o1,s3) name(o1) name(o2) name(o3) obj(o3,blue,triangle) obj(o1,blue,circle) obj(o2,blue,circle)
shape(0,s1,circle) shape(0,s2,circle) shape(0,s3,circle) color(0,s1,blue) color(0,s2,red) color(0,s3,blue) slot(s1) slot(s2) slot(s3) slot_name_comb(o2,s1) slot_name_comb(o3,s2) slot_name_comb(o1,s3) name(o1) name(o2) name(o3) obj(o1,blue,circle) obj(o2,blue,circle) obj(o3,red,circle)
shape(0,s1,circle) shape(0,s2,circle) shape(0,s3,circle) color(0,s1,blue) color(0,s2,blue) color(0,s3,blue) slot(s1) slot(s2) slot(s

### 3. encoding the background
Here we define two fixed objects and encode the last one as background.

This is the logic program.

In [6]:
ctl = clingo.Control(["0", "--warn=none"])
ctl.add("base", [], 
    """
    shape(0, s1, circle); shape(0, s1, triangle); shape(0, s1, rectangle) ; shape(0, s1, bg).
    shape(0, s2, circle); shape(0, s2, triangle); shape(0, s2, rectangle) ; shape(0, s2, bg).
    shape(0, s3, circle); shape(0, s3, triangle); shape(0, s3, rectangle) ; shape(0, s3, bg).
    
    color(0, s1, red); color(0, s1, blue); color(0, s1, green) ; color(0, s1, black).
    color(0, s2, red); color(0, s2, blue); color(0, s2, green) ; color(0, s2, black).
    color(0, s3, red); color(0, s3, blue); color(0, s3, green) ; color(0, s3, black).
    
    slot(s1).
    slot(s2).
    slot(s3).
    name(o1).
    name(o2).
    name(o3).

    %assign each name a slot
    {slot_name_comb(N,X): slot(X) }=1 :- name(N). %problem we have dublicated slots
    %remove each model which has multiple slots asigned to the same name
    :-  slot_name_comb(N1,X1), slot_name_comb(N2,X2), X1 == X2, N1 != N2. 

    %build the object ontop of the slot assignment
    object(N,C,S) :- color(0, X, C), shape(0, X, S), slot(X), name(N), slot_name_comb(N,X).
    

    %observations are provided during train time.
    :- not object(o1, green , triangle).
    :- not object(o2, green, circle).
    :- not object(o3, black, bg).
    """)

ctl.ground([("base", [])], context=Context())
out =ctl.solve(on_model=Context.on_model)


pp.pprint(ctl.statistics['summary']) #48 models why?

shape(0,s1,bg) shape(0,s2,circle) shape(0,s3,triangle) color(0,s1,black) color(0,s2,green) color(0,s3,green) slot(s1) slot(s2) slot(s3) slot_name_comb(o3,s1) slot_name_comb(o2,s2) slot_name_comb(o1,s3) name(o1) name(o2) name(o3) object(o1,green,triangle) object(o2,green,circle) object(o3,black,bg)
shape(0,s1,circle) shape(0,s2,bg) shape(0,s3,triangle) color(0,s1,green) color(0,s2,black) color(0,s3,green) slot(s1) slot(s2) slot(s3) slot_name_comb(o2,s1) slot_name_comb(o3,s2) slot_name_comb(o1,s3) name(o1) name(o2) name(o3) object(o1,green,triangle) object(o2,green,circle) object(o3,black,bg)
shape(0,s1,circle) shape(0,s2,triangle) shape(0,s3,bg) color(0,s1,green) color(0,s2,green) color(0,s3,black) slot(s1) slot(s2) slot(s3) slot_name_comb(o2,s1) slot_name_comb(o1,s2) slot_name_comb(o3,s3) name(o1) name(o2) name(o3) object(o1,green,triangle) object(o2,green,circle) object(o3,black,bg)
shape(0,s1,bg) shape(0,s2,triangle) shape(0,s3,circle) color(0,s1,black) color(0,s2,green) color(0,s3,g

# Test

In [7]:
ctl = clingo.Control(["0", "--warn=none"])
ctl.add("base", [], 
    """
    shape(0, s1, circle); shape(0, s1, triangle); shape(0, s1, rectangle) ; shape(0, s1, bg).
    shape(0, s2, circle); shape(0, s2, triangle); shape(0, s2, rectangle) ; shape(0, s2, bg).
    shape(0, s3, circle); shape(0, s3, triangle); shape(0, s3, rectangle) ; shape(0, s3, bg).
    
    color(0, s1, red); color(0, s1, blue); color(0, s1, green) ; color(0, s1, black).
    color(0, s2, red); color(0, s2, blue); color(0, s2, green) ; color(0, s2, black).
    color(0, s3, red); color(0, s3, blue); color(0, s3, green) ; color(0, s3, black).
    
    slot(s1).
    slot(s2).
    slot(s3).
    slot(s4).
    slot(s5).
    
    name(o1).
    name(o2).
    name(o3).
    name(o4).
    name(o5).

    %assign each name a slot
    {slot_name_comb(N,X): slot(X) }=1 :- name(N). %problem we have dublicated slots
    %remove each model which has multiple slots asigned to the same name
    :-  slot_name_comb(N1,X1), slot_name_comb(N2,X2), X1 == X2, N1 != N2. 

    %build the object ontop of the slot assignment
    object(N,C,S) :- color(0, X, C), shape(0, X, S), slot(X), name(N), slot_name_comb(N,X).
    

    %observations are provided during train time.
    :- not object(o1, green , triangle).
    :- not object(o2, green, circle).
    :- not object(o3, black, bg).
    """)
ctl.ground([("base", [])], context=Context())
out =ctl.solve(on_model=Context.on_model)



#pp.pprint(ctl.statistics['summary']) #48 models why?


shape(0,s1,circle) shape(0,s2,triangle) shape(0,s3,bg) color(0,s1,green) color(0,s2,green) color(0,s3,black) slot(s1) slot(s2) slot(s3) slot(s4) slot(s5) slot_name_comb(o2,s1) slot_name_comb(o1,s2) slot_name_comb(o3,s3) slot_name_comb(o5,s4) slot_name_comb(o4,s5) name(o1) name(o2) name(o3) name(o4) name(o5) object(o1,green,triangle) object(o2,green,circle) object(o3,black,bg)
shape(0,s1,circle) shape(0,s2,triangle) shape(0,s3,bg) color(0,s1,green) color(0,s2,green) color(0,s3,black) slot(s1) slot(s2) slot(s3) slot(s4) slot(s5) slot_name_comb(o2,s1) slot_name_comb(o1,s2) slot_name_comb(o3,s3) slot_name_comb(o4,s4) slot_name_comb(o5,s5) name(o1) name(o2) name(o3) name(o4) name(o5) object(o1,green,triangle) object(o2,green,circle) object(o3,black,bg)
shape(0,s1,bg) shape(0,s2,triangle) shape(0,s3,circle) color(0,s1,black) color(0,s2,green) color(0,s3,green) slot(s1) slot(s2) slot(s3) slot(s4) slot(s5) slot_name_comb(o3,s1) slot_name_comb(o1,s2) slot_name_comb(o2,s3) slot_name_comb(o5,s4) 

In [8]:
program = '''
1{color(0,s1,gray); color(0,s1,red); color(0,s1,blue); color(0,s1,green); color(0,s1,brown); color(0,s1,purple); color(0,s1,cyan); color(0,s1,yellow); color(0,s1,bg)}1.
1{color(0,s2,gray); color(0,s2,red); color(0,s2,blue); color(0,s2,green); color(0,s2,brown); color(0,s2,purple); color(0,s2,cyan); color(0,s2,yellow); color(0,s2,bg)}1.
1{color(0,s3,gray); color(0,s3,red); color(0,s3,blue); color(0,s3,green); color(0,s3,brown); color(0,s3,purple); color(0,s3,cyan); color(0,s3,yellow); color(0,s3,bg)}1.
1{color(0,s4,gray); color(0,s4,red); color(0,s4,blue); color(0,s4,green); color(0,s4,brown); color(0,s4,purple); color(0,s4,cyan); color(0,s4,yellow); color(0,s4,bg)}1.
1{color(0,s5,gray); color(0,s5,red); color(0,s5,blue); color(0,s5,green); color(0,s5,brown); color(0,s5,purple); color(0,s5,cyan); color(0,s5,yellow); color(0,s5,bg)}1.
1{color(0,s6,gray); color(0,s6,red); color(0,s6,blue); color(0,s6,green); color(0,s6,brown); color(0,s6,purple); color(0,s6,cyan); color(0,s6,yellow); color(0,s6,bg)}1.
1{color(0,s7,gray); color(0,s7,red); color(0,s7,blue); color(0,s7,green); color(0,s7,brown); color(0,s7,purple); color(0,s7,cyan); color(0,s7,yellow); color(0,s7,bg)}1.
1{color(0,s8,gray); color(0,s8,red); color(0,s8,blue); color(0,s8,green); color(0,s8,brown); color(0,s8,purple); color(0,s8,cyan); color(0,s8,yellow); color(0,s8,bg)}1.
1{color(0,s9,gray); color(0,s9,red); color(0,s9,blue); color(0,s9,green); color(0,s9,brown); color(0,s9,purple); color(0,s9,cyan); color(0,s9,yellow); color(0,s9,bg)}1.
1{color(0,s10,gray); color(0,s10,red); color(0,s10,blue); color(0,s10,green); color(0,s10,brown); color(0,s10,purple); color(0,s10,cyan); color(0,s10,yellow); color(0,s10,bg)}1.
1{shape(0,s1,cube); shape(0,s1,sphere); shape(0,s1,cylinder); shape(0,s1,bg)}1.
1{shape(0,s2,cube); shape(0,s2,sphere); shape(0,s2,cylinder); shape(0,s2,bg)}1.
1{shape(0,s3,cube); shape(0,s3,sphere); shape(0,s3,cylinder); shape(0,s3,bg)}1.
1{shape(0,s4,cube); shape(0,s4,sphere); shape(0,s4,cylinder); shape(0,s4,bg)}1.
1{shape(0,s5,cube); shape(0,s5,sphere); shape(0,s5,cylinder); shape(0,s5,bg)}1.
1{shape(0,s6,cube); shape(0,s6,sphere); shape(0,s6,cylinder); shape(0,s6,bg)}1.
1{shape(0,s7,cube); shape(0,s7,sphere); shape(0,s7,cylinder); shape(0,s7,bg)}1.
1{shape(0,s8,cube); shape(0,s8,sphere); shape(0,s8,cylinder); shape(0,s8,bg)}1.
1{shape(0,s9,cube); shape(0,s9,sphere); shape(0,s9,cylinder); shape(0,s9,bg)}1.
1{shape(0,s10,cube); shape(0,s10,sphere); shape(0,s10,cylinder); shape(0,s10,bg)}1.
1{material(0,s1,rubber); material(0,s1,metal); material(0,s1,bg)}1.
1{material(0,s2,rubber); material(0,s2,metal); material(0,s2,bg)}1.
1{material(0,s3,rubber); material(0,s3,metal); material(0,s3,bg)}1.
1{material(0,s4,rubber); material(0,s4,metal); material(0,s4,bg)}1.
1{material(0,s5,rubber); material(0,s5,metal); material(0,s5,bg)}1.
1{material(0,s6,rubber); material(0,s6,metal); material(0,s6,bg)}1.
1{material(0,s7,rubber); material(0,s7,metal); material(0,s7,bg)}1.
1{material(0,s8,rubber); material(0,s8,metal); material(0,s8,bg)}1.
1{material(0,s9,rubber); material(0,s9,metal); material(0,s9,bg)}1.
1{material(0,s10,rubber); material(0,s10,metal); material(0,s10,bg)}1.
1{size(0,s1,small); size(0,s1,large); size(0,s1,bg)}1.
1{size(0,s2,small); size(0,s2,large); size(0,s2,bg)}1.
1{size(0,s3,small); size(0,s3,large); size(0,s3,bg)}1.
1{size(0,s4,small); size(0,s4,large); size(0,s4,bg)}1.
1{size(0,s5,small); size(0,s5,large); size(0,s5,bg)}1.
1{size(0,s6,small); size(0,s6,large); size(0,s6,bg)}1.
1{size(0,s7,small); size(0,s7,large); size(0,s7,bg)}1.
1{size(0,s8,small); size(0,s8,large); size(0,s8,bg)}1.
1{size(0,s9,small); size(0,s9,large); size(0,s9,bg)}1.
1{size(0,s10,small); size(0,s10,large); size(0,s10,bg)}1.
slot(s1).
slot(s2).
slot(s3).
slot(s4).
slot(s5).
slot(s6).
slot(s7).
slot(s8).
slot(s9).
slot(s10).
name(o1).
name(o2).
name(o3).
name(o4).
name(o5).
name(o6).
name(o7).
name(o8).
name(o9).
name(o10).

:~ color(0,s1,gray). [2133, 0, 0]
:~ color(0,s1,red). [2518, 0, 1]
:~ color(0,s1,blue). [2232, 0, 2]
:~ color(0,s1,green). [2288, 0, 3]
:~ color(0,s1,brown). [2152, 0, 4]
:~ color(0,s1,purple). [2025, 0, 5]
:~ color(0,s1,cyan). [2159, 0, 6]
:~ color(0,s1,yellow). [2036, 0, 7]
:~ color(0,s1,bg). [2319, 0, 8]
:~ color(0,s2,gray). [2203, 1, 0]
:~ color(0,s2,red). [2127, 1, 1]
:~ color(0,s2,blue). [2373, 1, 2]
:~ color(0,s2,green). [1978, 1, 3]
:~ color(0,s2,brown). [2204, 1, 4]
:~ color(0,s2,purple). [2073, 1, 5]
:~ color(0,s2,cyan). [2236, 1, 6]
:~ color(0,s2,yellow). [2167, 1, 7]
:~ color(0,s2,bg). [2504, 1, 8]
:~ color(0,s3,gray). [2131, 2, 0]
:~ color(0,s3,red). [2150, 2, 1]
:~ color(0,s3,blue). [2277, 2, 2]
:~ color(0,s3,green). [2205, 2, 3]
:~ color(0,s3,brown). [2195, 2, 4]
:~ color(0,s3,purple). [2102, 2, 5]
:~ color(0,s3,cyan). [2189, 2, 6]
:~ color(0,s3,yellow). [2107, 2, 7]
:~ color(0,s3,bg). [2463, 2, 8]
:~ color(0,s4,gray). [2187, 3, 0]
:~ color(0,s4,red). [2157, 3, 1]
:~ color(0,s4,blue). [2302, 3, 2]
:~ color(0,s4,green). [2137, 3, 3]
:~ color(0,s4,brown). [2173, 3, 4]
:~ color(0,s4,purple). [2108, 3, 5]
:~ color(0,s4,cyan). [2158, 3, 6]
:~ color(0,s4,yellow). [2155, 3, 7]
:~ color(0,s4,bg). [2432, 3, 8]
:~ color(0,s5,gray). [2220, 4, 0]
:~ color(0,s5,red). [2089, 4, 1]
:~ color(0,s5,blue). [2225, 4, 2]
:~ color(0,s5,green). [2251, 4, 3]
:~ color(0,s5,brown). [2213, 4, 4]
:~ color(0,s5,purple). [1921, 4, 5]
:~ color(0,s5,cyan). [2352, 4, 6]
:~ color(0,s5,yellow). [2275, 4, 7]
:~ color(0,s5,bg). [2293, 4, 8]
:~ color(0,s6,gray). [2165, 5, 0]
:~ color(0,s6,red). [2529, 5, 1]
:~ color(0,s6,blue). [2263, 5, 2]
:~ color(0,s6,green). [2143, 5, 3]
:~ color(0,s6,brown). [2148, 5, 4]
:~ color(0,s6,purple). [1996, 5, 5]
:~ color(0,s6,cyan). [2108, 5, 6]
:~ color(0,s6,yellow). [2172, 5, 7]
:~ color(0,s6,bg). [2337, 5, 8]
:~ color(0,s7,gray). [2196, 6, 0]
:~ color(0,s7,red). [2176, 6, 1]
:~ color(0,s7,blue). [2290, 6, 2]
:~ color(0,s7,green). [2177, 6, 3]
:~ color(0,s7,brown). [2163, 6, 4]
:~ color(0,s7,purple). [1923, 6, 5]
:~ color(0,s7,cyan). [2191, 6, 6]
:~ color(0,s7,yellow). [2300, 6, 7]
:~ color(0,s7,bg). [2432, 6, 8]
:~ color(0,s8,gray). [2213, 7, 0]
:~ color(0,s8,red). [2211, 7, 1]
:~ color(0,s8,blue). [2268, 7, 2]
:~ color(0,s8,green). [2103, 7, 3]
:~ color(0,s8,brown). [2195, 7, 4]
:~ color(0,s8,purple). [1921, 7, 5]
:~ color(0,s8,cyan). [2257, 7, 6]
:~ color(0,s8,yellow). [2304, 7, 7]
:~ color(0,s8,bg). [2369, 7, 8]
:~ color(0,s9,gray). [2193, 8, 0]
:~ color(0,s9,red). [2248, 8, 1]
:~ color(0,s9,blue). [2307, 8, 2]
:~ color(0,s9,green). [2046, 8, 3]
:~ color(0,s9,brown). [2183, 8, 4]
:~ color(0,s9,purple). [2006, 8, 5]
:~ color(0,s9,cyan). [2198, 8, 6]
:~ color(0,s9,yellow). [2228, 8, 7]
:~ color(0,s9,bg). [2425, 8, 8]
:~ color(0,s10,gray). [2087, 9, 0]
:~ color(0,s10,red). [2374, 9, 1]
:~ color(0,s10,blue). [2245, 9, 2]
:~ color(0,s10,green). [2148, 9, 3]
:~ color(0,s10,brown). [2205, 9, 4]
:~ color(0,s10,purple). [2345, 9, 5]
:~ color(0,s10,cyan). [2152, 9, 6]
:~ color(0,s10,yellow). [1950, 9, 7]
:~ color(0,s10,bg). [2344, 9, 8]
:~ shape(0,s1,cube). [1534, 10, 0]
:~ shape(0,s1,sphere). [1497, 10, 1]
:~ shape(0,s1,cylinder). [1494, 10, 2]
:~ shape(0,s1,bg). [1089, 10, 3]
:~ shape(0,s2,cube). [1455, 11, 0]
:~ shape(0,s2,sphere). [1427, 11, 1]
:~ shape(0,s2,cylinder). [1525, 11, 2]
:~ shape(0,s2,bg). [1174, 11, 3]
:~ shape(0,s3,cube). [1515, 12, 0]
:~ shape(0,s3,sphere). [1480, 12, 1]
:~ shape(0,s3,cylinder). [1506, 12, 2]
:~ shape(0,s3,bg). [1105, 12, 3]
:~ shape(0,s4,cube). [1505, 13, 0]
:~ shape(0,s4,sphere). [1464, 13, 1]
:~ shape(0,s4,cylinder). [1532, 13, 2]
:~ shape(0,s4,bg). [1105, 13, 3]
:~ shape(0,s5,cube). [1486, 14, 0]
:~ shape(0,s5,sphere). [1458, 14, 1]
:~ shape(0,s5,cylinder). [1382, 14, 2]
:~ shape(0,s5,bg). [1237, 14, 3]
:~ shape(0,s6,cube). [1513, 15, 0]
:~ shape(0,s6,sphere). [1451, 15, 1]
:~ shape(0,s6,cylinder). [1432, 15, 2]
:~ shape(0,s6,bg). [1181, 15, 3]
:~ shape(0,s7,cube). [1552, 16, 0]
:~ shape(0,s7,sphere). [1559, 16, 1]
:~ shape(0,s7,cylinder). [1611, 16, 2]
:~ shape(0,s7,bg). [971, 16, 3]
:~ shape(0,s8,cube). [1489, 17, 0]
:~ shape(0,s8,sphere). [1471, 17, 1]
:~ shape(0,s8,cylinder). [1497, 17, 2]
:~ shape(0,s8,bg). [1135, 17, 3]
:~ shape(0,s9,cube). [1486, 18, 0]
:~ shape(0,s9,sphere). [1494, 18, 1]
:~ shape(0,s9,cylinder). [1595, 18, 2]
:~ shape(0,s9,bg). [1059, 18, 3]
:~ shape(0,s10,cube). [1509, 19, 0]
:~ shape(0,s10,sphere). [1497, 19, 1]
:~ shape(0,s10,cylinder). [1445, 19, 2]
:~ shape(0,s10,bg). [1140, 19, 3]
:~ material(0,s1,rubber). [1125, 20, 0]
:~ material(0,s1,metal). [1050, 20, 1]
:~ material(0,s1,bg). [1122, 20, 2]
:~ material(0,s2,rubber). [1104, 21, 0]
:~ material(0,s2,metal). [1044, 21, 1]
:~ material(0,s2,bg). [1150, 21, 2]
:~ material(0,s3,rubber). [1246, 22, 0]
:~ material(0,s3,metal). [949, 22, 1]
:~ material(0,s3,bg). [1122, 22, 2]
:~ material(0,s4,rubber). [1152, 23, 0]
:~ material(0,s4,metal). [992, 23, 1]
:~ material(0,s4,bg). [1159, 23, 2]
:~ material(0,s5,rubber). [1149, 24, 0]
:~ material(0,s5,metal). [1018, 24, 1]
:~ material(0,s5,bg). [1133, 24, 2]
:~ material(0,s6,rubber). [1216, 25, 0]
:~ material(0,s6,metal). [931, 25, 1]
:~ material(0,s6,bg). [1172, 25, 2]
:~ material(0,s7,rubber). [1150, 26, 0]
:~ material(0,s7,metal). [1035, 26, 1]
:~ material(0,s7,bg). [1113, 26, 2]
:~ material(0,s8,rubber). [1140, 27, 0]
:~ material(0,s8,metal). [1027, 27, 1]
:~ material(0,s8,bg). [1131, 27, 2]
:~ material(0,s9,rubber). [1211, 28, 0]
:~ material(0,s9,metal). [909, 28, 1]
:~ material(0,s9,bg). [1205, 28, 2]
:~ material(0,s10,rubber). [1121, 29, 0]
:~ material(0,s10,metal). [963, 29, 1]
:~ material(0,s10,bg). [1228, 29, 2]
:~ size(0,s1,small). [1123, 30, 0]
:~ size(0,s1,large). [1075, 30, 1]
:~ size(0,s1,bg). [1097, 30, 2]
:~ size(0,s2,small). [902, 31, 0]
:~ size(0,s2,large). [1158, 31, 1]
:~ size(0,s2,bg). [1271, 31, 2]
:~ size(0,s3,small). [1244, 32, 0]
:~ size(0,s3,large). [1057, 32, 1]
:~ size(0,s3,bg). [1008, 32, 2]
:~ size(0,s4,small). [1022, 33, 0]
:~ size(0,s4,large). [1051, 33, 1]
:~ size(0,s4,bg). [1234, 33, 2]
:~ size(0,s5,small). [1183, 34, 0]
:~ size(0,s5,large). [1053, 34, 1]
:~ size(0,s5,bg). [1063, 34, 2]
:~ size(0,s6,small). [1034, 35, 0]
:~ size(0,s6,large). [1191, 35, 1]
:~ size(0,s6,bg). [1076, 35, 2]
:~ size(0,s7,small). [1052, 36, 0]
:~ size(0,s7,large). [1081, 36, 1]
:~ size(0,s7,bg). [1164, 36, 2]
:~ size(0,s8,small). [959, 37, 0]
:~ size(0,s8,large). [1164, 37, 1]
:~ size(0,s8,bg). [1187, 37, 2]
:~ size(0,s9,small). [1174, 38, 0]
:~ size(0,s9,large). [988, 38, 1]
:~ size(0,s9,bg). [1142, 38, 2]
:~ size(0,s10,small). [899, 39, 0]
:~ size(0,s10,large). [1189, 39, 1]
:~ size(0,s10,bg). [1241, 39, 2]


%assign each name a slot
{slot_name_comb(N,X): slot(X) }=1 :- name(N). %problem we have dublicated slots
%remove each model which has multiple slots asigned to the same name
:-  slot_name_comb(N1,X1), slot_name_comb(N2,X2), X1 == X2, N1 != N2.


'''
'''

%build the object ontop of the slot assignment
object(N, S, M, P, C) :- size(0, X, S), material(0, X, M), shape(0, X, P), color(0, X, C), slot(X), name(N), slot_name_comb(N,X).
%define the SPNs
:- not object(o1, small, metal, cylinder, blue).
:- not object(o2, large, rubber, cylinder, blue).
:- not object(o3, large, metal, sphere, purple).
:- not object(o4, small, metal, cube, yellow).
:- not object(o5, large, metal, cylinder, gray).
:- not object(o6, small, metal, cube, cyan).
:- not object(o7, small, rubber, cylinder, cyan).
:- not object(o8, large, metal, cube, blue).
:- not object(o9, small, metal, cylinder, green).
:- not object(o10, small, rubber, cylinder, yellow).





'''

'\n\n%build the object ontop of the slot assignment\nobject(N, S, M, P, C) :- size(0, X, S), material(0, X, M), shape(0, X, P), color(0, X, C), slot(X), name(N), slot_name_comb(N,X).\n%define the SPNs\n:- not object(o1, small, metal, cylinder, blue).\n:- not object(o2, large, rubber, cylinder, blue).\n:- not object(o3, large, metal, sphere, purple).\n:- not object(o4, small, metal, cube, yellow).\n:- not object(o5, large, metal, cylinder, gray).\n:- not object(o6, small, metal, cube, cyan).\n:- not object(o7, small, rubber, cylinder, cyan).\n:- not object(o8, large, metal, cube, blue).\n:- not object(o9, small, metal, cylinder, green).\n:- not object(o10, small, rubber, cylinder, yellow).\n\n\n\n\n\n'

In [ ]:
ctl = clingo.Control(["0", "--warn=none"])
ctl.add("base", [], program)
ctl.ground([("base", [])], context=Context())
out =ctl.solve()

pp.pprint(ctl.statistics['summary'])



In [ ]:
'''
slot_name_comb(o2,s1),slot_name_comb(o1,s2) , slot_name_comb(o1,s1), slot_name_comb(o2,s2)  
we have both but want a unique assignment->
slot_name_comb(o2,s1),slot_name_comb(o1,s2) 
slot_name_comb(o1,s1), slot_name_comb(o2,s2)
'''


ctl = clingo.Control(["0", "--warn=none"])
ctl.add("base", [], 
    """
    shape(0, s1, circle); shape(0, s1, triangle).
    shape(0, s2, circle); shape(0, s2, triangle).

    color(0, s1, red); color(0, s1, blue).
    color(0, s2, red); color(0, s2, blue). 

    slot(s1).
    slot(s2).
    
    name(o1).
    name(o2).
    
    {slot_name_comb(N,X): slot(X) }=1 :- name(N). %problem we have dublicated slots
    :-  slot_name_comb(N1,X1), slot_name_comb(N2,X2), X1 == X2, N1 != N2. 
   
    obj(N,C,S) :- color(0, X, C), shape(0, X, S), slot(X), name(N), slot_name_comb(N,X).

    
    :- not obj(o1, blue,circle).
    :- not obj(o2, _,circle).
    
    
    """)
ctl.ground([("base", [])], context=Context())
out =ctl.solve(on_model=Context.on_model)

pp.pprint(ctl.statistics['summary']) 